In [66]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers

import numpy as np
from sklearn.model_selection import train_test_split

In [67]:
tf.__version__

'2.15.0'

# Data Pre-Processing

Open **play_style_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`. 

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves, the last move represents the play style (B[di] in this case)
    
We cropped only the moves to game list as:

In [68]:
df = open('./CSVs/Tutorial_play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [69]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [70]:
def prepare_input(moves):
    x = np.zeros((19,19,2))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[row,column,0] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,1] = 1
    return x

In [71]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 1145


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [72]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [73]:
x.shape

(1145, 19, 19, 2)

In [74]:
y.shape

(1145,)

In [75]:
np.bincount(y)

array([268, 530, 347])

Target is one-hot encoded and loss is changed to `categorical_crossentropy`

In [76]:
y_hot = tf.one_hot(y, depth=3)

Dataset splitting: 90% Training, 10% validation

In [77]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [90]:
def create_model():
    inputs = Input(shape=(19, 19, 2))
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)  
    outputs = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(3, activation='softmax', )(outputs)
    model = Model(inputs, outputs)
    opt = Adam(learning_rate=0.00005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [91]:
class GameSimulator:
    def __init__(self):
        # 初始游戏状态，你可以根据你的游戏规则初始化棋盘或游戏状态
        self.board = [[' ' for _ in range(19)] for _ in range(19)]  # 例如，这里创建一个19x19的空棋盘
    
    def reset(self):
        # 重置游戏状态为初始状态
        self.board = [[' ' for _ in range(19)] for _ in range(19)]  # 重新创建一个空棋盘作为初始状态
        return self.board  # 返回初始状态的游戏棋盘
    
    def get_legal_moves(self):
        legal_moves = []
        for i in range(19):
            for j in range(19):
                if self.board[i][j] == ' ':
                    legal_moves.append((i, j))  # 如果位置为空，表示合法的移动
        return legal_moves

    def make_move(self, move, player):
        row, col = move
        if self.board[row][col] == ' ':
            self.board[row][col] = player  # 将空位置设为玩家的标记，例如 'B' 或 'W'
            return True  # 返回 True 表示移动成功
        else:
            return False  # 返回 False 表示移动无效，位置已经被占用


In [92]:
model = create_model()
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 19, 19, 2)]       0         
                                                                 
 conv2d_63 (Conv2D)          (None, 19, 19, 32)        3168      
                                                                 
 batch_normalization_82 (Ba  (None, 19, 19, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_64 (Conv2D)          (None, 19, 19, 32)        50208     
                                                                 
 batch_normalization_83 (Ba  (None, 19, 19, 32)        128       
 tchNormalization)                                               
                                                                 
 conv2d_65 (Conv2D)          (None, 19, 19, 32)        2563

In [97]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 64,
    epochs = 20,
    validation_data=(x_val, y_val),
)

Epoch 1/20


17/17 [==============================] - 1s 34ms/step - loss: 0.0895 - accuracy: 0.9864 - val_loss: 1.0823 - val_accuracy: 0.6000
Epoch 2/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0959 - accuracy: 0.9883 - val_loss: 1.1796 - val_accuracy: 0.6174
Epoch 3/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0651 - accuracy: 0.9951 - val_loss: 1.1994 - val_accuracy: 0.6174
Epoch 4/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0600 - accuracy: 0.9942 - val_loss: 1.2057 - val_accuracy: 0.6174
Epoch 5/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0565 - accuracy: 0.9961 - val_loss: 1.2393 - val_accuracy: 0.6087
Epoch 6/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0577 - accuracy: 0.9942 - val_loss: 1.2093 - val_accuracy: 0.6522
Epoch 7/20
17/17 [==============================] - 1s 33ms/step - loss: 0.0555 - accuracy: 0.9942 - val_loss: 1.2178 - val_accuracy: 0.6435
Epoch 8/20
17/17 [======

In [98]:
model.save('./model_playstyle.h5')

/home/ttsai/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!